In [1]:
import numpy as np
import pandas as pd
import cv2
import ast
from skimage.feature import hog
from sklearn.decomposition import PCA
import ipyvolume as ipv
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm

In [2]:
def train_test_data(x_variable,y_variable,testsize,randomstate):
    x_train, x_test, y_train, y_test = train_test_split(x_variable,y_variable,test_size = testsize, random_state = randomstate)
    return x_train,x_test,y_train,y_test

In [3]:
def svm_training(Kernel,Degree,x_train,x_test,y_train,y_test):
    classifier = svm.SVC(kernel = Kernel,degree = Degree)
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return classifier,accuracy

# Training vehicle hog features via SVM using linear kernel......

In [4]:
vehicle_data = pd.read_csv('single_lane_Vehicle_hog.csv')

In [5]:
vehicle_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3771,3772,3773,3774,3775,3776,3777,3778,3779,MotorVehicle
0,0,0.369926,0.216900,0.095444,0.011491,0.038315,0.011491,0.012041,0.004760,0.026007,...,0.155598,0.028994,0.110025,0.028994,0.350096,0.127986,0.055012,0.000000,0.000000,1
1,1,0.124693,0.060799,0.098858,0.000000,0.045231,0.000000,0.000000,0.009195,0.000000,...,0.300639,0.265382,0.130717,0.195196,0.206488,0.154912,0.157947,0.300639,0.031436,1
2,2,0.353609,0.101121,0.063955,0.025280,0.101751,0.075841,0.047966,0.091324,0.139843,...,0.137770,0.016404,0.000000,0.000000,0.017260,0.014965,0.012227,0.076137,0.171552,1
3,3,0.331317,0.279131,0.005429,0.000000,0.024953,0.000000,0.002715,0.002146,0.000000,...,0.241216,0.008661,0.027660,0.006124,0.007747,0.010455,0.008217,0.019090,0.041913,1
4,4,0.188796,0.000000,0.008190,0.003237,0.005791,0.000000,0.002048,0.006475,0.000000,...,0.036492,0.023314,0.066352,0.169034,0.245017,0.000000,0.022117,0.000000,0.000000,1


In [6]:
vehicle_data = np.array(vehicle_data)

In [7]:
vehicle_hog_features = vehicle_data[:,1:-1]

In [8]:
vehicle_variables = vehicle_data[:,3781]

In [9]:
vehicle_x_train,vehicle_x_test,vehicle_y_train,vehicle_y_test = train_test_data(vehicle_hog_features,vehicle_variables,0.3,109)

In [10]:
vehicle_classifier,vehicle_accuracy = svm_training('linear',3,vehicle_x_train,vehicle_x_test,vehicle_y_train,vehicle_y_test)

In [11]:
vehicle_accuracy

0.9902439024390244

# Training helmet hog features via SVM using polynomial kernel......

In [12]:
helmet_data = pd.read_csv('single_lane_helmet_hog_features.csv')

In [13]:
helmet_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3771,3772,3773,3774,3775,3776,3777,3778,3779,Helmet
0,0,0.471778,0.027244,0.000000,0.0,0.012184,0.0,0.034461,0.054488,0.000000,...,0.489373,0.053429,0.011264,0.0,0.000000,0.0,0.022527,0.000000,0.000000,0
1,1,0.478791,0.035578,0.045003,0.0,0.000000,0.0,0.045003,0.035578,0.000000,...,0.480235,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1
2,2,0.494574,0.019872,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.076062,...,0.465382,0.031586,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.189318,0
3,3,0.512448,0.000000,0.012371,0.0,0.021869,0.0,0.000000,0.029341,0.018034,...,0.438008,0.033322,0.000000,0.0,0.005405,0.0,0.000000,0.012085,0.195399,0
4,4,0.352850,0.000000,0.110349,0.0,0.156058,0.0,0.110349,0.000000,0.000000,...,0.411617,0.000000,0.185272,0.0,0.037431,0.0,0.052935,0.000000,0.000000,1


In [14]:
helmet_data = np.array(helmet_data)

In [15]:
helmet_hog_features = helmet_data[:,1:-1]

In [16]:
helmet_variables = helmet_data[:,3781]

In [17]:
helmet_x_train,helmet_x_test,helmet_y_train,helmet_y_test = train_test_data(helmet_hog_features,helmet_variables,0.3,109)

In [18]:
helmet_classifier,helmet_accuracy = svm_training('poly',3,helmet_x_train,helmet_x_test,helmet_y_train,helmet_y_test)

In [19]:
helmet_accuracy

0.6821192052980133

# Using trained classifiers to predict the realtime output...........

In [20]:
def MotorVehicle_detection(vehicle,classifier):
    resized = cv2.resize(vehicle,(64,128),interpolation = cv2.INTER_AREA)
    f,i = hog(resized, orientations=9, pixels_per_cell=(8,8),cells_per_block=(2, 2),visualize = True)
    result = classifier.predict(f.reshape(1,-1))
    return result

In [21]:
def Helmet_detection(helmet_portion,classifier): 
    filter_img = cv2.GaussianBlur(helmet_portion,(5,5),100)
    resized = cv2.resize(filter_img,(64,128),interpolation = cv2.INTER_AREA)
    hf,_ = hog(resized, orientations=9, pixels_per_cell=(8,8),cells_per_block=(2, 2),visualize = True)
    result = classifier.predict([hf])
    return result

In [30]:
cap = cv2.VideoCapture('T2.mp4')
fgbg = cv2.createBackgroundSubtractorMOG2()
while(1):
    _,frame = cap.read()
    image = frame[0:frame.shape[0],0:int(frame.shape[1]-(frame.shape[1]*0.45))]
    #cv2.imshow('img',image)
    filter_img = cv2.GaussianBlur(image,(5,5),100)
    fgmask = fgbg.apply(filter_img)
    #_,thresh = cv2.threshold(fgmask,127,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for c in contours:
        area = cv2.contourArea(c)
        if 5000<area<10000:
            x,y,w,h = cv2.boundingRect(c)
            vehicle = image[(y+int(h*0.25)):(y+h),x:(x+w)]
            helmet_portion = vehicle[0:(0+int(h*0.25)),0:(0+w)]
            vehicle_class = MotorVehicle_detection(vehicle,vehicle_classifier)
            helmet_class = Helmet_detection(helmet_portion,helmet_classifier)
            if vehicle_class == 1:
                if helmet_class == 1:
                    color = (0,255,0)
                    text = 'bike with helmet'
                else:
                    color = (255,0,0)
                    text = 'bike without helmet'
            else:
                color = (255,255,255)
                text = 'not bike'
            cv2.putText(frame,text,(x,y), cv2.FONT_HERSHEY_SIMPLEX , 1,(0,0,255),1,cv2.LINE_AA)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.imshow('image',img)
    k = cv2.waitKey(5)
    if k == ord('s'):
        break
cap.release()
cv2.destroyAllWindows()